In [1]:
!nvidia-smi

Wed Jul 12 17:16:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q datasets diffusers transformers accelerate xformers tomesd torchmetrics[image]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.8/728.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import time
import contextlib
from torchmetrics.functional.multimodal import clip_score
from functools import partial


class Profile(contextlib.ContextDecorator):
    # YOLOv5 Profile class. Usage: @Profile() decorator or 'with Profile():' context manager
    def __init__(self, t=0.0):
        self.t = t
        self.cuda = torch.cuda.is_available()

    def __enter__(self):
        self.start = self.time()
        return self

    def __exit__(self, type, value, traceback):
        self.dt = self.time() - self.start  # delta-time
        self.t += self.dt  # accumulate dt

    def time(self):
        if self.cuda:
            torch.cuda.synchronize()
        return time.time()


clip_score_fn = partial(clip_score, model_name_or_path="openai/clip-vit-base-patch16")

def calculate_clip_score(images, prompts):
    images_int = (images * 255).astype("uint8")
    clip_score = clip_score_fn(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    return round(float(clip_score), 4)

In [4]:
from diffusers import DiffusionPipeline, StableDiffusionPipeline
from datasets import load_dataset

CONFIG = {
    "model_id": "runwayml/stable-diffusion-v1-5",
    "dtype": torch.float16,
    "gen": torch.manual_seed(0),
    "inference_steps": 25,
    "num_images_per_prompt": 4,
    "resolution": 512,
    "num_parti_prompts": 100,
    "challenge": "basic",
    "seed": 0,
    "tome_ratio": 0.5,
}

prompts = load_dataset("nateraw/parti-prompts", split="train")
prompts = prompts.shuffle()
prompts = [prompts[i]["Prompt"] for i in range(20)]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/nateraw___csv/nateraw--parti-prompts-03d8a21dfc9af5e7/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


In [5]:
pipeline = StableDiffusionPipeline.from_pretrained(CONFIG["model_id"], torch_dtype=CONFIG["dtype"])
pipeline.set_progress_bar_config(disable=True)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [6]:
pipeline.enable_model_cpu_offload()

dt = Profile()

with dt:
  results = pipeline(
      prompts, num_images_per_prompt=1, generator=CONFIG["gen"], output_type="numpy"
  )
print(f"{dt.t/len(prompts)} s")
print(f"CLIP score: {calculate_clip_score(results.images, prompts)}")
torch.cuda.empty_cache()

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
/usr/local/lib/python3.10/dist-packages/diffusers/image_processor.py:230: FutureWarning: the output_type numpy is outdated and has been set to `np`. Please make sure to set it to one of these instead: `pil`, `np`, `pt`, `latent`
  deprecate("Unsupported output_type", "1.0.0", deprecation_message, standard_warn=False)


7.14967577457428 s


CLIP score: 31.0135


In [7]:
pipeline.to("cuda")

It seems like you have activated model offloading by calling `enable_model_cpu_offload`, but are now manually moving the pipeline to GPU. It is strongly recommended against doing so as memory gains from offloading are likely to be lost. Offloading automatically takes care of moving the individual components vae, text_encoder, tokenizer, unet, scheduler, safety_checker, feature_extractor to GPU when needed. To make sure offloading works as expected, you should consider moving the pipeline back to CPU: `pipeline.to('cpu')` or removing the move altogether if you use offloading.


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.18.2",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

Original

In [8]:
dt = Profile()

with dt:
  results = pipeline(
      prompts, num_images_per_prompt=1, generator=CONFIG["gen"], output_type="numpy"
  )
print(f"{dt.t/len(prompts)} s")
print(f"CLIP score: {calculate_clip_score(results.images, prompts)}")
torch.cuda.empty_cache()

6.676596784591675 s
CLIP score: 31.998


Xformer

In [ ]:
pipeline.enable_xformers_memory_efficient_attention()

dt = Profile()

with dt:
  results = pipeline(
      prompts, num_images_per_prompt=1, generator=CONFIG["gen"], output_type="numpy"
  )
print(f"{dt.t/len(prompts)} s")
print(f"CLIP score: {calculate_clip_score(results.images, prompts)}")

pipeline.disable_xformers_memory_efficient_attention()
torch.cuda.empty_cache()

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


6.585616886615753 s


ToMe

In [ ]:
import tomesd

tomesd.apply_patch(pipeline, ratio=0.5)

dt = Profile()

with dt:
  image = pipeline(prompts, num_images_per_prompt=1, generator=CONFIG["gen"], output_type="numpy"
  )
print(f"{dt.t/len(prompts)} s")
print(f"CLIP score: {calculate_clip_score(results.images, prompts)}")

tomesd.remove_patch(pipeline)
torch.cuda.empty_cache()

XFormer + ToMe

In [ ]:
import tomesd

pipeline.enable_xformers_memory_efficient_attention()
tomesd.apply_patch(pipeline, ratio=CONFIG["tome_ratio"])

dt = Profile()

with dt:
  image = pipeline(prompts, num_images_per_prompt=1, generator=CONFIG["gen"], output_type="numpy"
  )
print(f"{dt.t/len(prompts)} s")
print(f"CLIP score: {calculate_clip_score(results.images, prompts)}")

torch.cuda.empty_cache()